In [47]:
import plotly.express as px

import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
from scipy.io.wavfile import read as wavread
def ToolReadAudio(cAudioFilePath):    
    [samplerate, x] = wavread(cAudioFilePath)    
    if x.dtype == 'float32':        
        audio = x    
    else:        
        # change range to [-1,1)        
        if x.dtype == 'uint8':            
            nbits = 8        
        elif x.dtype == 'int16':            
            nbits = 16        
        elif x.dtype == 'int32':            
            nbits = 32        
        audio = x / float(2**(nbits - 1))    
        # special case of unsigned format    
    if x.dtype == 'uint8':        
        audio = audio - 1.    
    return (samplerate, audio)

In [3]:
relevant_genres = ['funk', 'hiphop', 'jazz', 'rock']

import pandas as pd
info = pd.read_csv('./groove/info.csv')
#df=info[info['style'].isin(relevant_genres)]

#df.pivot(index='style', columns='beat_type', values='audio_filename')
df = pd.pivot_table(info, values='audio_filename', index=['style'],
                    columns=['beat_type'], aggfunc='count')
#df[df['beat'] >= 20]

info = info[info['style'].isin(relevant_genres)]
info = info[info['beat_type'] == "beat"]
info = info[info['audio_filename'].notna()].reset_index()
#locs = info['audio_filename'].to_numpy()


In [4]:
sr,audio = ToolReadAudio('./groove/'+'drummer1/session1/1_funk_80_beat_4-4.wav')
audio.shape[-1]

<ipython-input-2-a414e5f47eb1>:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  [samplerate, x] = wavread(cAudioFilePath)


2

In [48]:
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    # compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs   
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]    
    return (xb,t)

In [81]:
nsecs = 1
xb,t = block_audio(x,int(nsecs*sr),int(nsecs*sr/2),sr)

In [110]:
def get_novelty(xb):
    N = 2048#xb.shape[1]
    w = scipy.signal.hann(N)
    novelty=np.zeros([xb.shape[0],xb.shape[1]])
    for i in range(xb.shape[0]):
        xb_square = xb[i]**2
        energy_local = np.convolve(xb_square, w**2, 'same')
        energy_local_diff = np.diff(energy_local)
        energy_local_diff = np.concatenate((energy_local_diff, np.array([0])))
        novelty_energy = np.copy(energy_local_diff)
        novelty_energy[energy_local_diff < 0] = 0
        novelty[i] = novelty_energy
    return novelty
def get_num_onsets(novelty,thresh):
    onsets=np.zeros(novelty.shape[0])
    for i in range(novelty.shape[0]):
        p = scipy.signal.find_peaks(novelty[i])[0]
        onsets[i]=(novelty[i][p] >= thresh).sum()
    return onsets
def get_onset_data(xb,threshold):
    novelty=get_novelty(xb)
    onsets = get_num_onsets(novelty,threshold)
    return onsets#, (onsets/durationInSecs)


In [125]:
n=get_novelty(xb)
o=get_num_onsets(n,0.001)

In [126]:
get_onset_data(xb,0.001)

array([0., 0., 0., 1., 2., 2., 2., 1., 1., 3., 3., 2., 3., 2., 1., 2., 1.,
       1., 3., 2., 1., 3., 2., 1., 3., 3., 1., 2., 3., 2., 3., 4., 2., 3.,
       3., 2., 3., 3., 2., 3., 4., 3., 3., 2., 1., 3., 3., 2., 1., 1., 2.,
       3., 3., 2., 2., 1., 1., 2., 3., 3., 2., 2., 2., 3., 3., 2., 3., 2.,
       1., 3., 3., 2., 3., 3., 2., 2., 2., 2., 2., 1., 1., 3., 3., 2., 3.,
       3., 2., 2., 2., 2., 2., 1., 0., 2., 4., 2., 2., 2., 1., 3., 4., 2.,
       2., 2., 0., 3., 5., 3., 3., 3., 1., 2., 4., 3., 3., 4., 2., 2., 4.,
       3., 3., 4., 3., 3., 4., 3., 3., 3., 2., 3., 3., 2., 3., 3., 2., 3.,
       3., 2., 3., 3., 2., 3., 3., 2., 3., 3., 2., 2., 3., 4., 4., 3., 2.,
       3., 3., 2., 3., 3., 2., 3., 3., 2., 2., 1., 1., 3., 3., 2., 3., 2.,
       1., 3., 5., 3.])

In [127]:
np.mean(o)

2.3850574712643677

In [128]:
np.std(o)

0.9321867426293056

In [44]:
info['audio'] = ''
for index, row in info.iterrows():
    #if index % 10 == 0:
        #print(index)
    try:
        sr,audio = ToolReadAudio('./groove/'+row['audio_filename'])
        if audio.shape[-1] == 2:
            info['audio'][index] = np.mean(audio,axis=1)
        else:
            info['audio'][index] = audio
        #info['audio'][index] = audio
    except Exception as e: # Some files are unable to be read by scipy's wavfile read, so skipping them and removing the corresponding rows
        info.drop(index,inplace=True)
        #print(e)
        
        print("Error, dropping row")
        print(f"size of info is now: {len(info)}")
info.reset_index(drop=True, inplace=True)
info.drop('index',axis=1,inplace=True)

<ipython-input-42-c4b24d39d73e>:8: WavFileWarning:

Chunk (non-data) not understood, skipping it.

<ipython-input-44-771846dd385f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-44-771846dd385f>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Error, dropping row
size of info is now: 250
Error, dropping row
size of info is now: 249
Error, dropping row
size of info is now: 248
Error, dropping row
size of info is now: 247
